In [1]:
"""
Script to extract text data from Mason petrology reports for load into SA_geodata. Input file path to the pdf 
documents and it will return an excel speadsheet containing the report no, sample no and description.
"""

'\nScript to extract text data from Mason petrology reports for load into SA_geodata. Input file path to the pdf \ndocuments and it will return an excel speadsheet containing the report no, sample no and description.\n'

In [2]:
import os
from pdf2image import convert_from_path, convert_from_bytes
import tempfile
import shutil
import re
import cv2 
import numpy as np
import pytesseract
from pytesseract import Output
import pandas as pd
pytesseract.pytesseract.tesseract_cmd = 'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'
from timeit import default_timer as timer
from datetime import datetime

In [3]:
#pdf_path = os.path.normpath(str(input(f'Please input the full path to the pdf files:')))
pdf_path = os.path.normpath('C:\\Users\\dutchr01\\MyPyScripts\\Mason_pet_extract\\Mason_text_ext')

pdf_list = []
with os.scandir(pdf_path) as it:
    for entry in it:
        if entry.name.endswith(".pdf") and entry.is_file():
            pdf_list.append(entry.path)

print(pdf_list)

['C:\\Users\\dutchr01\\MyPyScripts\\Mason_pet_extract\\Mason_text_ext\\3286.pdf', 'C:\\Users\\dutchr01\\MyPyScripts\\Mason_pet_extract\\Mason_text_ext\\3294.pdf', 'C:\\Users\\dutchr01\\MyPyScripts\\Mason_pet_extract\\Mason_text_ext\\3347.pdf', 'C:\\Users\\dutchr01\\MyPyScripts\\Mason_pet_extract\\Mason_text_ext\\3363.pdf', 'C:\\Users\\dutchr01\\MyPyScripts\\Mason_pet_extract\\Mason_text_ext\\3792.pdf', 'C:\\Users\\dutchr01\\MyPyScripts\\Mason_pet_extract\\Mason_text_ext\\3807.pdf', 'C:\\Users\\dutchr01\\MyPyScripts\\Mason_pet_extract\\Mason_text_ext\\3997.pdf', 'C:\\Users\\dutchr01\\MyPyScripts\\Mason_pet_extract\\Mason_text_ext\\3998.pdf', 'C:\\Users\\dutchr01\\MyPyScripts\\Mason_pet_extract\\Mason_text_ext\\4065.pdf', 'C:\\Users\\dutchr01\\MyPyScripts\\Mason_pet_extract\\Mason_text_ext\\4069.pdf', 'C:\\Users\\dutchr01\\MyPyScripts\\Mason_pet_extract\\Mason_text_ext\\4109.pdf', 'C:\\Users\\dutchr01\\MyPyScripts\\Mason_pet_extract\\Mason_text_ext\\4114.pdf', 'C:\\Users\\dutchr01\\MyPyS

In [4]:

start = timer()


report_dict = {'Report_No':[], 'Sample_No':[], 'Description':[]}
rock_type_dict = {'Sample_No':[], 'Rock_Type':[]}

# config for TesseractOCR
custom_config = r'--oem 3 --psm 6'

#create a temp directory and Convert pdf pages to individual jpg images for OCR and image extraction

temp_dir = tempfile.TemporaryDirectory()
f_name = ''


for pdf in pdf_list:
    jpg_path_list = []
    OCR_text_lst = []
    f_name = pdf.split('\\')[-1]
    images = convert_from_path(pdf, dpi = 300, output_folder=temp_dir.name,fmt='jpg')
    
# close PIL image objects
    for im in images:
        im.close()

    print(f'working on {f_name}')

#scan a file directory and return a list of jpg files to iterate over
    with os.scandir(temp_dir.name) as it:
        for entry in it:
            if entry.name.endswith(".jpg") and entry.is_file():
                jpg_path_list.append(entry.path)

    for im_path in jpg_path_list:
        image = cv2.imread(im_path, 0)
        d = pytesseract.image_to_string(image, config=custom_config)
        OCR_text_lst.append(d)

    #Grab the footer text from page 2 of pdf
    footer = OCR_text_lst[1].split('\n')
    footer[-1]

    #generate the regex pattern to remove the footer from pages
    start = footer[-1].split()[0] +' '+ footer[-1].split()[1]
    end = footer[-1].split()[-2] +' '+ footer[-1].split()[-1]
    footer_pattern = (f'{start}(.*?){end}')
    
    #generate a single string from the list of strings 
    text_string = ''
    raw_text = text_string.join(OCR_text_lst)
    raw_text

    #find and replace footer with white space
    text_c1 = re.sub(footer_pattern,'',raw_text)
    
    #Find the idx of the first instance of 'SAMPLE' used as a sample page headder, to  remove preceeding intro pages
    petro = text_c1.find('PETROGRAPHIC')
    first_sample = text_c1[petro:].find('SAMPLE') + petro
    
    # new string only contining the samples and removing all multiple newlines but adding extra newline at section headdings, then splitting into individual samples
    
    text_c2 = re.sub(r"\n+", "\n", text_c1[first_sample:])
    text_c2 = re.sub(r"(?<=\w)\.\n(?=\w)",".\n\n", text_c2)
    #text_c2 = re.sub(r"(?<=\w)\n(?=\w)"," ", text_c2,) #This will grab \n in text and replace w wht space but messes with mineral lists

    text_c2 = text_c2.replace('SAMPLE','***SAMPLE').replace('HOLE','\nHOLE').replace('LITH','\nLITH').replace('COMMENT','\nCOMMENT').replace('SECTION','\nSECTION').replace('HAND','\nHAND').replace('FIG','\nFIG').replace('ROCK','\nROCK').replace('PETROGRAPHY','\nPETROGRAPHY').replace('Mineral','\nMineral').replace('In thin section','\nIn thin section').replace('INTERPRETATION','\nINTERPRETATION').replace('SOURCE','\nSOURCE').replace('OTHER','\nOTHER').replace('STRAT/\nLITH', '\nSTRAT/LITH').replace('APPENDIX','\nAPPENDIX').replace('BRIEF','\nBRIEF').replace('In polished thin','\nIn polished thin').replace('CLIENT','\nCLIENT').replace('FIELD','\nFIELD').replace('GA NO','\nGA NO').split('***')

   #remove empty strings from list
    descriptions = [x for x in text_c2 if x]
    
    fig_pattern = '(FIG|Fig)'
    sample_no_list =[]
    sample_no_pattern = '(\d{7})|(R\d{7})'
    fig_list = []
    rock_pattern = r"(?s)ROCK N.*?:(.*?)PETRO"
    rock_list = []
    for l in descriptions:
        m = re.search(sample_no_pattern,l)
        f = re.search(fig_pattern,l)
        roc = re.search(rock_pattern,l,)
        if m:
            sample_no_list.append(m.group().strip('R'))
        if f:
            fig_list.append(m.group().strip('R'))
        if roc:
            rock_list.append(roc.group(1))

    print(f'lengths are equal :{len(descriptions) == len(sample_no_list)}')

    report_list = []
    for i in range(len(descriptions)):
        report_list.append(f_name)

    report_dict['Report_No'].append(report_list)
    report_dict['Sample_No'].append(sample_no_list)
    report_dict['Description'].append(descriptions)
    rock_type_dict['Sample_No'].append(sample_no_list)
    rock_type_dict['Rock_Type'].append(rock_list)

    #remove images from previous loop
    with os.scandir(temp_dir.name) as it:
        for entry in it:
            if entry.name.endswith(".jpg") and entry.is_file():
                os.remove(entry)



working on 3286.pdf
lengths are equal :True
working on 3294.pdf
lengths are equal :True
working on 3347.pdf
lengths are equal :True
working on 3363.pdf
lengths are equal :True
working on 3792.pdf
lengths are equal :True
working on 3807.pdf
lengths are equal :True
working on 3997.pdf
lengths are equal :True
working on 3998.pdf
lengths are equal :True
working on 4065.pdf
lengths are equal :True
working on 4069.pdf
lengths are equal :True
working on 4109.pdf
lengths are equal :True
working on 4114.pdf
lengths are equal :True
working on 4224.pdf
lengths are equal :True
working on 4235.pdf
lengths are equal :True
working on MG3792.pdf
lengths are equal :True
working on MG3807.pdf
lengths are equal :True
working on MG3819.pdf
lengths are equal :True
working on MG3919.pdf
lengths are equal :True
working on MG3920.pdf
lengths are equal :True
working on MG3997.pdf
lengths are equal :True
working on MG4051.pdf
lengths are equal :True
working on MG4058.pdf
lengths are equal :True
working on MG418

In [5]:
#print(text_c2)

In [85]:
# Cell to verify all reports have correct number of samples and descriptions, if not they can be dropped from export
length_chek = []
index_list =[]
for i, (a,b,c,l,k) in enumerate(zip(report_dict['Report_No'],report_dict['Sample_No'],report_dict['Description'],rock_type_dict['Sample_No'],rock_type_dict['Rock_Type'])):
    length_chek.append([a[0],len(a),len(b),len(c),len(l),len(k)])
    if len(b)!=len(c) or len(l)!=len(k):
        index_list.append(i)


length_chek

[['3286.pdf', 10, 10, 10, 10, 10],
 ['3294.pdf', 17, 13, 17, 13, 11],
 ['3347.pdf', 29, 25, 29, 25, 9],
 ['3363.pdf', 26, 22, 26, 22, 9],
 ['3792.pdf', 21, 21, 21, 21, 21],
 ['3807.pdf', 17, 17, 17, 17, 17],
 ['3997.pdf', 47, 47, 47, 47, 47],
 ['3998.pdf', 9, 9, 9, 9, 9],
 ['4065.pdf', 20, 20, 20, 20, 20],
 ['4069.pdf', 7, 7, 7, 7, 7],
 ['4109.pdf', 26, 26, 26, 26, 26],
 ['4114.pdf', 2, 2, 2, 2, 2],
 ['4224.pdf', 67, 67, 67, 67, 67],
 ['4235.pdf', 2, 2, 2, 2, 2],
 ['MG3792.pdf', 21, 21, 21, 21, 21],
 ['MG3807.pdf', 17, 17, 17, 17, 17],
 ['MG3819.pdf', 33, 33, 33, 33, 33],
 ['MG3919.pdf', 8, 8, 8, 8, 8],
 ['MG3920.pdf', 24, 24, 24, 24, 24],
 ['MG3997.pdf', 47, 47, 47, 47, 47],
 ['MG4051.pdf', 23, 23, 23, 23, 23],
 ['MG4058.pdf', 6, 6, 6, 6, 6],
 ['MG4188.pdf', 32, 32, 32, 32, 32],
 ['MG4196.pdf', 4, 4, 4, 4, 4],
 ['MG4224.pdf', 67, 67, 67, 67, 67]]

In [86]:
index_list

[1, 2, 3]

In [89]:
df_all = pd.DataFrame(report_dict)
df_all

,Report_No,Sample_No,Description
0,"[3286.pdf, 3286.pdf, 3286.pdf, 3286.pdf, 3286....","[1039227, 1039229, 1039230, 1039232, 1039233, ...","[SAMPLE : R1039227 (Pitjantjatjara Supersuite,..."
1,"[3294.pdf, 3294.pdf, 3294.pdf, 3294.pdf, 3294....","[1473471, 1473484, 1473485, 1473489, 1473491, ...","[SAMPLE : 1473471 (Mt Painter Inlier, SA)\n\nS..."
2,"[3347.pdf, 3347.pdf, 3347.pdf, 3347.pdf, 3347....","[1564585, 1564582, 1564575, 1565711, 1565712, ...",[SAMPLE : R1564585 (Michael Hills Mafic/Ultram...
3,"[3363.pdf, 3363.pdf, 3363.pdf, 3363.pdf, 3363....","[1565714, 1565715, 1565717, 1565722, 1565724, ...","[SAMPLE R# : R1565714 (Harcus Intrusion, Giles..."
4,"[3792.pdf, 3792.pdf, 3792.pdf, 3792.pdf, 3792....","[1842253, 1842254, 1842255, 1842256, 1842257, ...","[SAMPLE ID : 1842253 (Tieyon 1:100,000 Map She..."
5,"[3807.pdf, 3807.pdf, 3807.pdf, 3807.pdf, 3807....","[1834087, 1834090, 1834091, 1834092, 1834093, ...","[SAMPLE NO. : 1834087 (Fowler Map Sheet, South..."
6,"[3997.pdf, 3997.pdf, 3997.pdf, 3997.pdf, 3997....","[2012525, 2012537, 2012539, 2012548, 2012549, ...","[SAMPLE ID : 2012525 (Musgrave Province, South..."
7,"[3998.pdf, 3998.pdf, 3998.pdf, 3998.pdf, 3998....","[1851423, 1900382, 2053760, 2054871, 2054872, ...","[SAMPLE NO. : 1851423\nDRILL \nHOLE,m_— : SAE6..."
8,"[4065.pdf, 4065.pdf, 4065.pdf, 4065.pdf, 4065....","[2066659, 2066660, 2066661, 2066647, 2066648, ...","[SAMPLE R# : R2066659 (Emmie Bluff Prospect, G..."
9,"[4069.pdf, 4069.pdf, 4069.pdf, 4069.pdf, 4069....","[1998158, 1998160, 1998163, 2016125, 2018616, ...",[SAMPLE NO. : 1998158 (South Australia)\n\nOTH...


In [94]:
df_failed = df_all.loc[index_list]

,Report_No,Sample_No,Description
1,"[3294.pdf, 3294.pdf, 3294.pdf, 3294.pdf, 3294....","[1473471, 1473484, 1473485, 1473489, 1473491, ...","[SAMPLE : 1473471 (Mt Painter Inlier, SA)\n\nS..."
2,"[3347.pdf, 3347.pdf, 3347.pdf, 3347.pdf, 3347....","[1564585, 1564582, 1564575, 1565711, 1565712, ...",[SAMPLE : R1564585 (Michael Hills Mafic/Ultram...
3,"[3363.pdf, 3363.pdf, 3363.pdf, 3363.pdf, 3363....","[1565714, 1565715, 1565717, 1565722, 1565724, ...","[SAMPLE R# : R1565714 (Harcus Intrusion, Giles..."


In [96]:
# create dataframe and export to excel
df = pd.DataFrame(report_dict).drop(index_list).apply(pd.Series.explode)

df2 = pd.Series(fig_list)

df_failed = df_all.loc[index_list]

df_rock = pd.DataFrame(rock_type_dict).drop(index_list).apply(pd.Series.explode)

now = datetime.now().strftime("%Y%m%d_%H%M")

with pd.ExcelWriter(pdf_path+'\\'+now+'_Mason_petrology_export.xlsx') as writer:
    df.to_excel(writer, sheet_name = 'extracted_pet_descriptions', index=False)
    df_failed.to_excel(writer, sheet_name = 'Failed_extract', index=False)
    df2.to_excel(writer, sheet_name='Samples with figs', index=False)

with pd.ExcelWriter(pdf_path+'\\'+now+'_Rock_types.xlsx') as writer:
    df_rock.to_excel(writer, sheet_name = 'Mason_rock_types', index=False)




In [84]:
fig_list

[]